# Create Features from the application data model using Datasets

In the examples below, we create features using the `WindTurbine` application data model via the Pandas APIs provided from *C3 AI Datasets*.
The `WindTurbine.eval()` API is used to retrieve data. After retrieving the data, we resample to an hourly interval,
and create a `Feature.Set` and associated `Feature`s from the engineered features. Finally, we materialize and then retrieve the
feature data.

We have three types of interest:

* WindTurbine - this has a row for each wind turbine and is the subject type on which we will create features.
* WindTurbineMeasurementSeries - this is a "time series" header type which holds a time series for each wind turbine. This level of the data model is not needed for time series features created via Datasets. We include it here to enable features created via metrics in the same application.
* WindTurbineMeasurement - this contains the actual time series data

Rather than access the measurement series and measurement data directly, we will access it through the `WindTurbine` type.
This allows the feature store infrastructure to add and remove subject filters as needed.

## 1. Retrieve, denormalize, and resample the raw time series data

To create features, we first need to retrieve the raw time series data for the fields we want (the `activePower` and `gearOilTemperature` fields of
`WindTurbineMeasurements` in this case).

In order to create features with a subject type of `WindTurbine`, we need to run our `eval()` call from that type.
In doing so, one can reference the fields from `WindTurbineMeasurement` by using the path `turbineMeasurements.data.FIELD_NAME`.

In the `eval()` call, we specify a filter to only look at `demo_TURBINE-1`. This will make our feature engineering
faster. When the actual feature data is materialized, this "subject filter" is removed.

We then call `flatten(level=2)` to denormalize the result into a flat data frame.

Finally, we will resample the unaligned data to an hourly interval.

In [1]:
proj='id, turbineMeasurements.data.start, turbineMeasurements.data.activePower, turbineMeasurements.data.gearOilTemperature'
df = c3.WindTurbine.eval(projection=proj,
                         filter="id=='demo_TURBINE-1'")\
                   .flatten(level=2)
df.head()

subject  ... turbineMeasurements_data_gearOilTemperature.value
0  demo_TURBINE-1  ...                                              55.0
1  demo_TURBINE-1  ...                                              66.4
2  demo_TURBINE-1  ...                                              76.7
3  demo_TURBINE-1  ...                                              57.7
4  demo_TURBINE-1  ...                                              71.1

[5 rows x 4 columns]

By default, `eval` returns a `subject` column containing the Wind Turbine id for each row. Other columns are named based on the
projection expressions. We want to rename `turbineMeasurements_data_start.value` to `timestamp` so that the feature
store knows this is the column indicating the time value for each sample. We also want to provide more convenient names
for our features.

After renaming our columns, we change the types of the columns to match our desired types.
We need to do this because `flatten()` call returns Data with all columns as type `object`.

In [2]:
df = df.rename(columns={'turbineMeasurements_data_start.value':'timestamp',
                        'turbineMeasurements_data_activePower.value':'activePower',
                        'turbineMeasurements_data_gearOilTemperature.value':'gearOilTemperature'})\
       .astype({'timestamp':'datetime64[ns]', 'activePower':'float64', 'gearOilTemperature':'float64'})
df.head()

subject           timestamp  activePower  gearOilTemperature
0  demo_TURBINE-1 2022-01-01 00:00:02       5109.0                55.0
1  demo_TURBINE-1 2022-01-01 00:09:12       4828.0                66.4
2  demo_TURBINE-1 2022-01-01 01:58:39       4845.0                76.7
3  demo_TURBINE-1 2022-01-01 03:11:51       4762.0                57.7
4  demo_TURBINE-1 2022-01-01 05:52:41       5362.0                71.1

Now, we resample to an hourly interval. We fill missing data with the most recent value so that
rolling feature calculations work correctly.

We will be updating the columns in our data frame, so we call `toDataRef()` to make it update-able.

In [3]:
df = df.groupby('subject').resample(rule='H', on='timestamp').mean().reset_index().fillna(method='ffill').toDataRef()
df.head()

subject           timestamp  activePower  gearOilTemperature
0  demo_TURBINE-1 2022-01-01 00:00:00       4968.5                60.7
1  demo_TURBINE-1 2022-01-01 01:00:00       4845.0                76.7
2  demo_TURBINE-1 2022-01-01 02:00:00       4845.0                76.7
3  demo_TURBINE-1 2022-01-01 03:00:00       4762.0                57.7
4  demo_TURBINE-1 2022-01-01 04:00:00       4762.0                57.7

## 2. Compute features

We can now compute our features based on this resampled time series data. We engineer a selection of features sensitive to spikes in the measurements.
For more context on these features, see the notebook `Tutorials/Use-Cases/Wind-Turbine-Predictive-Maintenance.ipynb`.

For each source data column (corresponding to a sensor), we compute:

* A rolling maximum of the difference in values using a window of size 24 (1 day) and min_periods = 1. To achieve this you would need to chain `diff()`, `rolling()` with `max()` aggregation.
* A rolling standard deviation of values with a window of size 24 and min_periods = 1 using `rolling()` with `std()` aggregation.
* A rolling mean of values with a window of size 24 and min_periods = 1 using `rolling()` with `mean()` aggregation.
* An absolute deviation from the rolling median of values with a window of size 96 and min_periods = 1 using `rolling()` with `median()` aggregation, then `.abs()` to get just the absolute value of this deviation.

Note that we can easily create several features in a loop, saving us the work of manually defining each feature indvidually.

In [4]:
sensors = ["activePower",  "gearOilTemperature"]
for sensor in sensors:
        # Create a diff column smoothed by rolling max
        df[sensor + '_diff'] = df[sensor].diff().rolling(window=24, min_periods=1).max()
    
        # Create a rolling standard deviation column
        df[sensor + '_rolling_std'] = df[sensor].rolling(window=24, min_periods=1).std()
            # Create a rolling mean column
        df[sensor + '_rolling_mean'] = df[sensor].rolling(window=24, min_periods=1).mean()
    
        # Create a deviation from a rolling median column
        df[sensor + '_median_deviation'] = (df[sensor] - df[sensor].rolling(window=96, min_periods=1).median()).abs()

df.head()    

subject  ... gearOilTemperature_median_deviation
0  demo_TURBINE-1  ...                                 0.0
1  demo_TURBINE-1  ...                                 8.0
2  demo_TURBINE-1  ...                                 0.0
3  demo_TURBINE-1  ...                                11.0
4  demo_TURBINE-1  ...                                 3.0

[5 rows x 12 columns]

Drop the original data values, since we only need the computed features.

In [5]:
df = df.drop(sensors, axis=1)
df.head()

subject  ... gearOilTemperature_median_deviation
0  demo_TURBINE-1  ...                                 0.0
1  demo_TURBINE-1  ...                                 8.0
2  demo_TURBINE-1  ...                                 0.0
3  demo_TURBINE-1  ...                                11.0
4  demo_TURBINE-1  ...                                 3.0

[5 rows x 10 columns]

We now have our final engineered feature data. Show the final columns we have in our `Data` instance.

In [6]:
df.infer_dtypes

subject                                        object
timestamp                              datetime64[ns]
activePower_diff                              float64
activePower_rolling_std                       float64
activePower_rolling_mean                      float64
activePower_median_deviation                  float64
gearOilTemperature_diff                       float64
gearOilTemperature_rolling_std                float64
gearOilTemperature_rolling_mean               float64
gearOilTemperature_median_deviation           float64
dtype: object

## 3. Create and materialize a feature set

We can now create a feature set and the underlying features via `upsertFeatureSet`.
We specify `materialize=False`, as we want to do that separately. Since we know the interval of these
features is hourly, we also set that on the feature set.

In [7]:
fs = df.upsertFeatureSet(name='windTurbinePandasFeatures', subjectType=c3.WindTurbine, materialize=False, interval='HOUR')
fs.get() # show the full feature set definition

{
  "type" : "Feature.Set",
  "id" : "WindTurbine#windTurbinePandasFeatures",
  "name" : "windTurbinePandasFeatures",
  "meta" : {
    "appCode" : 1764276653578211516,
    "env" : "c3",
    "app" : "wt23",
    "created" : "2023-04-26T22:07:19Z",
    "createdBy" : "BA",
    "updated" : "2023-04-26T22:07:19Z",
    "updatedBy" : "BA",
    "timestamp" : "2023-04-26T22:07:19Z",
    "fetchInclude" : "[]",
    "fetchType" : "Feature.Set"
  },
  "version" : 1,
  "subjectType" : "WindTurbine",
  "interval" : "HOUR",
  "features" : [ "gearOilTemperature_median_deviation", "gearOilTemperature_rolling_mean", "activePower_median_deviation", "activePower_rolling_std", "gearOilTemperature_diff", "gearOilTemperature_rolling_std", "activePower_rolling_mean", "activePower_diff" ]
}

We materialize the feature set. This will materialize for all subjects, not just for `demo_TURBINE-1`.

In [8]:
fs.materialize(sync=True)

## 4. Evaluate feature data

To retrieve feature data from the feature store, you can use `WindTurbine.evalFeatureSetBatch`.
First, we look at Turbine 1, which was used in our feature engineering.

In [9]:
eval_feature_result1 = c3.WindTurbine.evalFeatureSetBatch(filter="id=='demo_TURBINE-1'", featureSet=fs)
eval_feature_result1.head()

subject  ... gearOilTemperature_rolling_std
0  demo_TURBINE-1  ...                            NaN
1  demo_TURBINE-1  ...                      11.313708
2  demo_TURBINE-1  ...                       9.237604
3  demo_TURBINE-1  ...                      10.177590
4  demo_TURBINE-1  ...                       9.934787

[5 rows x 10 columns]

Now, we look at Turbine 2, which was not included in the original feature engineering, but was included in materialization.

In [10]:
eval_feature_result2 = c3.WindTurbine.evalFeatureSetBatch(filter="id=='demo_TURBINE-2'", featureSet=fs)
eval_feature_result2.head()

subject  ... gearOilTemperature_rolling_std
0  demo_TURBINE-2  ...                      13.985194
1  demo_TURBINE-2  ...                      17.915259
2  demo_TURBINE-2  ...                      19.194244
3  demo_TURBINE-2  ...                      20.287164
4  demo_TURBINE-2  ...                      22.015707

[5 rows x 10 columns]

You can also use `eval()` to retrieve features. We list the features we want on the `projection` parameter. The
default interval for `eval()` is `'DAY'`, so we explicitly provide the desired interval of `'HOUR'`.

In [11]:
eval_result = c3.WindTurbine.eval(filter=True, projection='activePower_diff, activePower_median_deviation', interval='HOUR')
eval_result.head()

subject  ... activePower_median_deviation
0  demo_TURBINE-1  ...                         0.00
1  demo_TURBINE-1  ...                        61.75
2  demo_TURBINE-1  ...                         0.00
3  demo_TURBINE-1  ...                        83.00
4  demo_TURBINE-1  ...                        83.00

[5 rows x 4 columns]